In [1]:
import json
from time import time

In [21]:
file = "../data/2017-01"
with open(file, 'rb') as inputfile:
    data = []
    for line in inputfile.readlines()[:100000]:
        line = line.decode('utf-8')
        line = json.loads(line)
        data.append(line['body'])
#         print(line)
#         break

In [17]:
data

['OMGosh im an idiot this week.  Thank you',
 'Why did you post the same thing 4 days ago, but remove the link you posted then? \n\nhttps://www.reddit.com/r/Bitcoin/comments/5khpwn/double_your_btc/',
 '&gt; female viewport\n\nTopkek',
 'If you had an API on for example Poloniex and another site or two that feature alts you could run automatic arb using another coin as a medium to quickly move your capital back and forth. \n\nNow your bot is going to get more complex and you have to watch for slippage as the small coins can be moved easily with large buy/sell orders. ',
 '[removed]',
 'What you explained is pretty much the definition of the world deflationary, thus rendering your post redundant.\n',
 "I believe that bitcoin is highly undervalued, and that the bubble phase hasn't even remotely begun. $5000 by this year is my prediction.",
 'See, you\'re still asserting my ignorance of the meaning but not actually justifying that assertion. Tell you what, go on over to enwp.org/marxism wh

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
import sklearn
sklearn.__version__

'0.19.1'

In [6]:
n_features = 1000
n_components = 10
n_top_words = 20

In [21]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')

In [22]:
tf = tf_vectorizer.fit_transform(data)

In [38]:
lda = LatentDirichletAllocation(n_topics = 20, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)

In [27]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=10, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [37]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [30]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: coins yes alts trade watch playing sell file keys poloniex halving exactly 10 wallet waiting isn aspect reasonably millions small
Topic #1: video did ll use maybe guy 12 getting thing instead supload bitcoin views different shitcoin days try post indicator look
Topic #2: new year wallet power happy paper years comment bitcoin 2017 great hd backup solo miners gains yeah stop post pool
Topic #3: make trust removed gt doesn does community 1000 choose bank vote central like cash set point company monero hardware mean
Topic #4: bitcoin com https like address http just people yes right www client account buy using live business cash day network
Topic #5: thank mentioned support buy easy term short bitfinex taking obviously know worse don generally ny knows options fine indicator china
Topic #6: bitcoin check thanks using hodl post 2017 month big run 5000 etf pretty january got buy solar social doesn state
Topic #7: bitcoin just people don btc like know think time really use money g

In [34]:
lda.perplexity(tf)

1344281.8965174572

In [35]:
lda.score(tf)

-140619.8072567581

## Grid Search on LDA 

In [10]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

/Users/yize/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yize/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [39]:
parameters = {'max_iter': (5, 10, 20, 50), 'n_topics': (5, 10, 15, 20)}

In [47]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
lda = LatentDirichletAllocation(learning_method='online', learning_offset=50.,random_state=0)

In [49]:
pipeline = Pipeline([
    ('tf_vectorizer', tf_vectorizer),
    ('lda', lda)
])

In [50]:
grid_search = GridSearchCV(pipeline, parameters)
grid_search.fit(data)

ValueError: Invalid parameter max_iter for estimator Pipeline. Check the list of available parameters with `estimator.get_params().keys()`.

In [18]:
len(data)

156850

In [31]:
perp = []

In [38]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

In [39]:
pre = time()
for n_topics in range(3, 4):
    lda = LatentDirichletAllocation(n_components = 3, max_iter=10, learning_method='online', learning_offset=50.,random_state=0)
    lda.fit(tf)
    perp.append(lda.perplexity(tf))
    print("{} topics is done, perp is {}".format(n_topics, lda.perplexity(tf)))
    print_top_words(lda, tf_feature_names, n_top_words)
#     break
#     print(lda.score(tf))

print("time cost: {}".format(time()-pre))

3 topics is done, perp is 563.265881344866
Topic #0: bitcoin gt don people just like block segwit think transactions transaction does way core use miners network need know make
Topic #1: com https deleted wallet reddit www bitcoin http monero coinbase comments amp use address gt private btc key org np
Topic #2: bitcoin just price like btc money people buy don time think good market going long right china know really sell

time cost: 184.89830875396729


In [29]:
len(data)

100000

In [ ]:
1 topics is done, perp is 609.233541742176
2 topics is done, perp is 578.7028423557113
3 topics is done, perp is 563.265881344866
4 topics is done, perp is 573.4052257054898
5 topics is done, perp is 587.4237787092175
6 topics is done, perp is 589.9777281906931
7 topics is done, perp is 630.5905259277095
8 topics is done, perp is 627.3084500229427
9 topics is done, perp is 646.6673450989138
10 topics is done, perp is 652.2983967811948
11 topics is done, perp is 686.0986079152194
12 topics is done, perp is 662.227270393487
13 topics is done, perp is 692.5523522598721
14 topics is done, perp is 702.2042842850713
15 topics is done, perp is 722.9144140654396
16 topics is done, perp is 723.8353136452766
17 topics is done, perp is 746.9076008547785
18 topics is done, perp is 748.9445188359059
19 topics is done, perp is 761.7871778582253
time cost: 7345.377376079559

In [40]:
perp

[587.42377870921746,
 589.97772819069314,
 630.59052592770945,
 627.30845002294268,
 646.66734509891376,
 652.29839678119481,
 686.09860791521942,
 662.22727039348695,
 692.55235225987212,
 702.20428428507125,
 722.91441406543959,
 723.83531364527664,
 746.90760085477848,
 748.94451883590591,
 761.78717785822528,
 609.23354174217604,
 578.70284235571125,
 563.26588134486599,
 573.40522570548978,
 563.26588134486599]